In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
from prophet import Prophet

from config.config_data import MAIN_FILE
from src.data_preprocessing.data_loader import time_split
from src.modeling.evaluation import smape, mae

In [ ]:
DATA_DIR = (
    Path("..")
    / ".."
    / ".."
    / "hfactory_magic_folders"
    / "plastic_cost_prediction"
    / "data"
)

In [ ]:
df = pd.read_csv(DATA_DIR / MAIN_FILE)

In [ ]:
ts = df[["time", "best_price_compound"]]  # .dropna()
ts["time"] = pd.to_datetime(ts["time"])
ts.columns = ["ds", "y"]

In [ ]:
spl = time_split(ts)
sMAPE_total, MAE_total = 0.0, 0.0

# iterate over cv folds
for train_idx, test_idx in spl:
    train = ts.iloc[train_idx]
    test = ts.iloc[test_idx]

    m = Prophet(
        changepoint_prior_scale=0.5, seasonality_mode="multiplicative"
    )  #
    m.fit(train)
    future = m.make_future_dataframe(periods=9, freq="MS")
    forecast = m.predict(future)
    m.plot(forecast)
    plt.plot(test["ds"], test["y"], ".", color="red")

    # predictions are constant of last observation in train data
    preds = forecast["yhat"].tail(9).iloc[2::3].to_numpy()

    # get avg. sMAPE and MAE over 3, 6, 9 months on test data for fold
    sMAPE = smape(test.iloc[2::3, 1].to_numpy(), preds)
    MAE = mae(test.iloc[2::3, 1].to_numpy(), preds)
    sMAPE_total += sMAPE
    MAE_total += MAE
    print(sMAPE)

# average MAE and sMAPE over folds
avg_sMAPE = sMAPE_total / len(spl)
avg_MAE = MAE_total / len(spl)

In [ ]:
avg_sMAPE